# Machine Learning Major Project

In this project I use PCA to show that unfortunately biologically realistic data and models are still seperable.

Rationale I use python with dask and numba to save time, although Julia perhaps better long term solution, although I use that elsewhere.

If biologically realistic models where a better imitation of real experimental cells, then data and models would not be seperable, by plotting onto a high dimensional feature space we show that data and models are seperable.

We then use random forest regression to find out which dimensions explain the most variance.

In [1]:
import sys    
print('Running on Python version: {}'.format(sys.version))
import matplotlib.pyplot as plt
plt.show()

''' None of these worked that well.
import dask_ml
from dask_ml.decomposition import PCA
dir(dask_ml.utils)
from dask_ml.datasets import make_classification
from dask_ml.wrappers import Incremental
from sklearn.linear_model import SGDClassifier
'''
import os
import dask.dataframe as dd    
# Common stuff to import
%matplotlib inline
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
sns.set(font_scale=1.5)
import pandas as pd
os.getcwd()

Running on Python version: 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]


/usr/local/lib/python3.5/dist-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


'/home/russell/safe2/neuronunit/neuronunit/examples'

In [2]:
#help(df)

with open('dm_on_models.p','rb') as f:
    (RAW_dtc,ADEXP_dtc) = pickle.load(f)
# Impute remaining missing values with 0.
#

/home/russell/safe2/neuronunit/neuronunit/optimisation/optimization_management.py:8: UserWarning: X11 plotting backend not available, consider installing
  warnings.warn('X11 plotting backend not available, consider installing')


consider installing pynn a heavier backend


INFO       Cache size for target "cython": 4171 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "/home/russell/.cython/brian_extensions" directory. [brian2]


pynn python Error


In [3]:
# Special stuff to import
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE

In [4]:
cwd = os.getcwd()
path2data = os.path.join(cwd,'data')

In [5]:
standard = 0
strong = 0
easy_map = [
            {'AP12AmplitudeDropTest':standard},
            {'AP1SSAmplitudeChangeTest':standard},
            {'AP1AmplitudeTest':standard},
            {'AP1WidthHalfHeightTest':standard},
            {'AP1WidthPeakToTroughTest':standard},
            {'AP1RateOfChangePeakToTroughTest':standard},
            {'AP1AHPDepthTest':standard},
            {'AP2AmplitudeTest':standard},
            {'AP2WidthHalfHeightTest':standard},
            {'AP2WidthPeakToTroughTest':standard},
            {'AP2RateOfChangePeakToTroughTest':standard},
            {'AP2AHPDepthTest':standard},
            {'AP12AmplitudeChangePercentTest':standard},
            {'AP12HalfWidthChangePercentTest':standard},
            {'AP12RateOfChangePeakToTroughPercentChangeTest':standard},
            {'AP12AHPDepthPercentChangeTest':standard},
            {'InputResistanceTest':str('ir_currents')},
            {'AP1DelayMeanTest':standard},
            {'AP1DelaySDTest':standard},
            {'AP2DelayMeanTest':standard},
            {'AP2DelaySDTest':standard},
            {'Burst1ISIMeanTest':standard},
            {'Burst1ISISDTest':standard},
            {'InitialAccommodationMeanTest':standard},
            {'SSAccommodationMeanTest':standard},
            {'AccommodationRateToSSTest':standard},
            {'AccommodationAtSSMeanTest':standard},
            {'AccommodationRateMeanAtSSTest':standard},
            {'ISICVTest':standard},
            {'ISIMedianTest':standard},
            {'ISIBurstMeanChangeTest':standard},
            {'SpikeRateStrongStimTest':strong},
            {'AP1DelayMeanStrongStimTest':strong},
            {'AP1DelaySDStrongStimTest':strong},
            {'AP2DelayMeanStrongStimTest':strong},
            {'AP2DelaySDStrongStimTest':strong},
            {'Burst1ISIMeanStrongStimTest':strong},
            {'Burst1ISISDStrongStimTest':strong},
        ]



# Load and clean data in one cell

In [6]:
# Open the 1.5x rheobase file
filename = os.path.join(cwd,'onefive_df.pkl')
with open(filename, 'rb') as f:
    df = pickle.load(f)

    # A function to convert all cells containing array (or other things) into floats.  
def f(x):
    try:
        return np.mean(x)
    except:
        try:
            return np.mean(x['pred'])
        except:
            print(x)
            raise e
df = df.fillna(0)

# Apply this function to each dataframe in order to convert all cells into floats.
# Also call fillna() first to impute missing values with 0's.  
df = df.fillna(0).applymap(f)
#df_30x = df_30x.fillna(0).applymap(f)
df.head()

df
df_dask = dd.from_pandas(df.T, npartitions=8)#, chunksize=8)    
#df_30x = df_30x.fillna(0).applymap(f)

#for i in df.columns:
#    print(i)
    
#dir(df_dask)#.persist()#.compute()


In [7]:
% time df_dask = df_dask.fillna(0).applymap(f)

CPU times: user 5.37 s, sys: 20 ms, total: 5.39 s
Wall time: 5.38 s


In [8]:
dm_labels = [list(keys.keys())[0] for keys in easy_map ]
keep_columns = []
for l in easy_map:
    for k in l.keys():
        if str(k)+str('_3.0x') in df.columns:
            keep_columns.append(str(k)+str('_3.0x'))
        elif str(k)+str('_1.5x') in df.columns:
            keep_columns.append(str(k)+str('_1.5x'))

df1 = pd.DataFrame(df,columns=keep_columns)
df = df1
df.head()

,AP12AmplitudeDropTest_3.0x,AP1SSAmplitudeChangeTest_3.0x,AP1AmplitudeTest_3.0x,AP1WidthHalfHeightTest_3.0x,AP1WidthPeakToTroughTest_3.0x,AP1RateOfChangePeakToTroughTest_3.0x,AP1AHPDepthTest_3.0x,AP2AmplitudeTest_3.0x,AP2WidthHalfHeightTest_3.0x,AP2WidthPeakToTroughTest_3.0x,...,ISICVTest_3.0x,ISIMedianTest_3.0x,ISIBurstMeanChangeTest_3.0x,SpikeRateStrongStimTest_3.0x,AP1DelayMeanStrongStimTest_3.0x,AP1DelaySDStrongStimTest_3.0x,AP2DelayMeanStrongStimTest_3.0x,AP2DelaySDStrongStimTest_3.0x,Burst1ISIMeanStrongStimTest_3.0x,Burst1ISISDStrongStimTest_3.0x
313861539,0.009156,0.005390,0.037937,0.120,0.460,-0.182133,0.045844,0.028781,0.155,0.56,...,0.311657,10.235,-98.819898,47.0,22.710,0.0,607.405,1.136868e-13,295.7975,5.684342e-14
313861677,0.001219,0.000000,0.064875,0.685,10.035,-0.009834,0.033813,0.063656,0.785,0.00,...,2.516959,66.200,15.416833,3.0,53.410,0.0,103.550,0.000000e+00,54.0050,0.000000e+00
313862167,-0.000125,-0.000290,0.018000,0.235,1.915,-0.032050,0.043375,0.018125,0.250,0.00,...,10.784342,51.390,8.857865,19.0,52.225,0.0,93.770,0.000000e+00,43.3850,0.000000e+00
313862167,-0.000125,-0.000290,0.018000,0.235,1.915,-0.032050,0.043375,0.018125,0.250,0.00,...,10.784342,51.390,8.857865,19.0,52.225,0.0,93.770,0.000000e+00,43.3850,0.000000e+00
313862167,58.906250,33.244049,58.906250,0.200,0.000,0.000000,0.000000,0.000000,0.000,1.55,...,0.000000,0.000,0.000000,52.0,30.980,0.0,44.965,0.000000e+00,14.4050,0.000000e+00


In [9]:
df.shape

(7285, 38)

In [10]:
for l in easy_map:
    for k in l.keys():
        if str(k)+str('_1.5x') in df.columns:
            df.rename(columns={str(k)+str('_1.5x'):str(k)}, inplace=True)

for l in easy_map:
    for k in l.keys():
        if str(k)+str('_3.0x') in df.columns: 
            df.rename(columns={str(k)+str('_3.0x'):str(k)}, inplace=True)


for k in RAW_dtc.keys():
    temp0 = pd.DataFrame(RAW_dtc[k][0].dm_test_features)
    sub_df = df[temp0.columns]
    break
    

cnt=0
for k in RAW_dtc.keys():
    temp0 = pd.DataFrame(RAW_dtc[k][0].dm_test_features)
    temp0.rename(index={0:str(k)+str(' fe_izhi')},inplace=True)
    temp1 = pd.DataFrame(ADEXP_dtc[k][0].dm_test_features)
    temp1.rename(index={0:str(k)+str(' adexp')},inplace=True)
    #display(temp1)
    #sub_df = sub_df.append(temp0)
    #sub_df = sub_df.append(temp1)
    
    sub_df = pd.concat([temp0, sub_df])#, ignore_index=True)
    sub_df = pd.concat([temp1, sub_df])#, ignore_index=True)

    cnt+=1

df = sub_df

In [11]:
sub_df.head()
size=cnt*2
print(size)
#RAW_dtc.keys()

10


In [12]:
#df = df[any(keep_columns) in df]
df



,AP12AHPDepthPercentChangeTest,AP12AmplitudeChangePercentTest,AP12AmplitudeDropTest,AP12HalfWidthChangePercentTest,AP12RateOfChangePeakToTroughPercentChangeTest,AP1AHPDepthTest,AP1AmplitudeTest,AP1DelayMeanStrongStimTest,AP1DelayMeanTest,AP1DelaySDStrongStimTest,...,Burst1ISIMeanTest,Burst1ISISDStrongStimTest,Burst1ISISDTest,ISIBurstMeanChangeTest,ISICVTest,ISIMedianTest,InitialAccommodationMeanTest,InputResistanceTest,SSAccommodationMeanTest,SpikeRateStrongStimTest
Cerebellum Purkinje cell adexp,0.000000,NaN,0.000000,NaN,-0.000000,0.054625,NaN,2.000,5.000,0.0,...,2.0000,0.000000e+00,0.000000e+00,0.000000e+00,7.204293,2.000,-29.292929,3.071018e-05,-100.000000,275.0
Cerebellum Purkinje cell fe_izhi,NaN,NaN,NaN,NaN,NaN,0.040037,0.044963,13.845,26.065,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,-100.000000,5.021916e-02,-100.000000,10.0
Neocortex pyramidal cell layer 5-6 adexp,0.000000,NaN,0.000000,NaN,-0.000000,0.117375,NaN,3.000,7.000,0.0,...,4.0000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,4.000,-24.242424,3.071024e-05,-100.000000,269.5
Neocortex pyramidal cell layer 5-6 fe_izhi,0.470129,0.019662,-0.000014,0.000000e+00,1.593721,0.052092,0.069881,9.465,30.155,0.0,...,85.0250,0.000000e+00,0.000000e+00,1.349831e+00,367.008073,85.600,-20.000000,6.107706e-02,0.000000,33.5
Hippocampus CA1 basket cell adexp,0.000000,NaN,0.000000,NaN,-0.000000,0.117375,NaN,3.000,7.000,0.0,...,4.0000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,4.000,-24.242424,3.071024e-05,-100.000000,269.5
Hippocampus CA1 basket cell fe_izhi,11.827433,-0.016312,0.000011,7.092199e-01,57.381143,0.030769,0.069998,10.920,24.060,0.0,...,97.9700,0.000000e+00,0.000000e+00,-4.753126e+00,91.982119,95.575,0.000000,2.416693e-02,0.000000,34.5
Hippocampus CA1 pyramidal cell adexp,0.000000,NaN,0.000000,NaN,-0.000000,0.117375,NaN,3.000,7.000,0.0,...,4.0000,0.000000e+00,0.000000e+00,0.000000e+00,NaN,4.000,-24.242424,3.071024e-05,-100.000000,269.5
Hippocampus CA1 pyramidal cell fe_izhi,0.470129,0.019662,-0.000014,0.000000e+00,1.593721,0.052092,0.069881,9.465,30.155,0.0,...,85.0250,0.000000e+00,0.000000e+00,1.349831e+00,367.008073,85.600,-20.000000,6.107706e-02,0.000000,33.5
Olfactory bulb (main) mitral cell adexp,0.000000,NaN,0.000000,NaN,-0.000000,0.117375,NaN,3.000,7.000,0.0,...,3.0000,0.000000e+00,0.000000e+00,-7.579123e-12,33.105946,4.000,-23.232323,7.558388e-05,-100.000000,269.5
Olfactory bulb (main) mitral cell fe_izhi,18.353319,0.027752,-0.000019,4.065041e+00,-31.291293,0.040589,0.069970,9.450,22.335,0.0,...,75.2525,0.000000e+00,0.000000e+00,1.885270e+01,32.419386,81.190,0.000000,2.548990e-02,0.000000,43.0


In [13]:
#import dask.dataframe as dd

df.iloc[0][192:195]


if 'InputResistanceTest_1.5x' in df.columns:
    # in order to find out what is seperating and what is not.
    del df['InputResistanceTest_1.5x']
    del df['InputResistanceTest_3.0x']
    del df['ohmic_input_resistance_1.5x']
    del df['ohmic_input_resistance_3.0x']
    del df['time_1.5x']                              
    #       0.190362
    del df['decay_time_constant_after_stim_3.0x']
    del df['voltage_deflection_3.0x']
    del df['steady_state_hyper_3.0x']
    del df['steady_state_voltage_stimend_3.0x']
    del df['voltage_deflection_vb_ssse_3.0x']
    del df['sag_amplitude_3.0x']
    #0.198310
    del df['is_not_stuck_1.5x']
    del df['AHP_depth_abs_1.5x']



In [14]:
df.tail()

,AP12AHPDepthPercentChangeTest,AP12AmplitudeChangePercentTest,AP12AmplitudeDropTest,AP12HalfWidthChangePercentTest,AP12RateOfChangePeakToTroughPercentChangeTest,AP1AHPDepthTest,AP1AmplitudeTest,AP1DelayMeanStrongStimTest,AP1DelayMeanTest,AP1DelaySDStrongStimTest,...,Burst1ISIMeanTest,Burst1ISISDStrongStimTest,Burst1ISISDTest,ISIBurstMeanChangeTest,ISICVTest,ISIMedianTest,InitialAccommodationMeanTest,InputResistanceTest,SSAccommodationMeanTest,SpikeRateStrongStimTest
NMLCL001594,-21.797548,-6.175600,5.680001,6.849315,-20.011535,19.595903,91.974881,3.20,9.89,0.0,...,18.105,0.0,0.000000e+00,1.952036,32.115224,21.90,0.0,71.106876,-5.263158,90.0
NMLCL001650,-0.006635,-0.044739,0.024598,0.131579,-0.031287,30.003234,54.980529,98.75,195.23,0.0,...,151.100,0.0,2.842171e-14,0.026476,10961.079423,151.12,0.0,64.812843,0.000000,10.5
NMLCL001657,0.051921,-0.089518,0.053719,0.171527,-3.028343,34.082421,60.009550,66.31,144.93,0.0,...,219.200,0.0,5.684342e-14,0.091283,3269.344983,219.30,0.0,38.619367,0.000000,9.5
NMLCL001660,0.824537,-0.431151,0.039358,0.000000,-99.961023,96.750154,9.128643,59.35,129.85,0.0,...,142.215,0.0,0.000000e+00,39.802715,12.077925,164.77,0.0,21.387796,0.000000,17.5
NMLCL001663,-0.006635,-0.044739,0.024598,0.131579,-0.031287,30.003234,54.980529,98.75,195.23,0.0,...,151.100,0.0,2.842171e-14,0.026476,10961.079423,151.12,0.0,64.812843,0.000000,10.5


In [15]:
# Join the dataframes horizonstally, so that all features coming from df_15x get a '_1.5x' suffix
# and all the ones from df_30x get a '_3.0x' suffix
#df = df_15x.join(df_30x, lsuffix='_1.5x', rsuffix='_3.0x')
def f(x):
    try:
        return np.mean(x)
    except:
        try:
            return np.mean(x['pred'])
        except:
            print(x)
            raise e
df = df.fillna(0)

In [16]:
print("There are %d models+data and %d features" % df.shape)
#print("There are %d models+data and %d features" % dask_frame.shape)

There are 7295 models+data and 38 features


# The data frame is big.
*lets experiment with dask -lazy pandas array to avoid storing it all in memory, and to do CPU-block-mapped operations in partitions 

In [17]:
#%%capture
# Impute remaining missing values with 0.

dask_frame = dd.from_pandas(df.T,npartitions=8)
dask_frame.size
dask_frame.drop_duplicates

<bound method _Frame.drop_duplicates of Dask DataFrame Structure:
                              Cerebellum Purkinje cell adexp Cerebellum Purkinje cell fe_izhi Neocortex pyramidal cell layer 5-6 adexp Neocortex pyramidal cell layer 5-6 fe_izhi Hippocampus CA1 basket cell adexp Hippocampus CA1 basket cell fe_izhi Hippocampus CA1 pyramidal cell adexp Hippocampus CA1 pyramidal cell fe_izhi Olfactory bulb (main) mitral cell adexp Olfactory bulb (main) mitral cell fe_izhi 313861539 313861677 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 313862167 31386216

In [18]:
df.size;
df.drop_duplicates;

In [19]:

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df[:] = ss.fit_transform(df.values)
df.groupby(df.index).first()
df = pd.DataFrame.drop_duplicates(df)
print(len(df))

1420


In [20]:
# Turn all features into Normal(0,1) variables
# Important since features all have different scales
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
df[:] = ss.fit_transform(df.values)
df.groupby(df.index).first()
df = pd.DataFrame.drop_duplicates(df)
print(len(df))



1420


In [21]:
#df.index[0:50]

In [22]:
# Check to see what is there.  Might also check to see if there is data there.
#df_30x#.head()

# make model dataframe

model_idx2 = df.head(7).index.tolist()#  list(range(0,9))#idx for idx in df.index.values if type(idx)==str]
model_no_trans_df2 = df[df.index.isin(model_idx2)]
model_no_trans_df2.index.name = 'OptCells'
model_df2 = model_no_trans_df2.copy()
model_df2.index.name = 'OptCells'

model_idx = [idx for idx in df.index.values if type(idx)==str]
model_no_trans_df = df[df.index.isin(model_idx)]
model_no_trans_df.index.name = 'Cell_ID'
model_df = model_no_trans_df.copy()
model_df.index.name = 'Cell_ID'

# make experiment dataframe
experiment_idx = [idx for idx in df.index.values if type(idx)==int]
experiment_no_trans_df = df[df.index.isin(experiment_idx)]
experiment_df = experiment_no_trans_df.copy()
#print(len(experiment_df))
model_df2
model_no_trans_df2

,AP12AHPDepthPercentChangeTest,AP12AmplitudeChangePercentTest,AP12AmplitudeDropTest,AP12HalfWidthChangePercentTest,AP12RateOfChangePeakToTroughPercentChangeTest,AP1AHPDepthTest,AP1AmplitudeTest,AP1DelayMeanStrongStimTest,AP1DelayMeanTest,AP1DelaySDStrongStimTest,...,Burst1ISIMeanTest,Burst1ISISDStrongStimTest,Burst1ISISDTest,ISIBurstMeanChangeTest,ISICVTest,ISIMedianTest,InitialAccommodationMeanTest,InputResistanceTest,SSAccommodationMeanTest,SpikeRateStrongStimTest
OptCells,,,,,,,,,,,,,,,,,,,,,
Cerebellum Purkinje cell adexp,-0.034168,-0.044056,-0.170394,-0.061382,0.318542,-1.161653,-1.527886,-0.337510,-0.393407,-0.120591,...,-0.601711,-0.15436,-0.176145,-0.083516,-0.061837,-0.697797,0.005640,-0.873915,-0.437490,7.083295
Cerebellum Purkinje cell fe_izhi,-0.034168,-0.044056,-0.170394,-0.061382,0.318542,-1.163067,-1.526587,-0.262662,-0.035224,-0.120591,...,-0.623455,-0.15436,-0.176145,-0.083516,-0.084995,-0.711988,-0.513731,-0.873789,-0.437490,-0.511590
Neocortex pyramidal cell layer 5-6 adexp,-0.034168,-0.044056,-0.170394,-0.061382,0.318542,-1.155569,-1.527886,-0.331191,-0.359399,-0.120591,...,-0.579966,-0.15436,-0.176145,-0.083516,-0.084995,-0.683607,0.042738,-0.873915,-0.437490,6.925665
Neocortex pyramidal cell layer 5-6 fe_izhi,-0.033224,-0.044054,-0.170395,-0.061382,0.404431,-1.161898,-1.525867,-0.290339,0.034321,-0.120591,...,0.300952,-0.15436,-0.176145,-0.081320,1.094775,-0.104630,0.073900,-0.873761,0.170559,0.161919
Hippocampus CA1 basket cell fe_izhi,-0.010420,-0.044058,-0.170392,-0.060397,3.410929,-1.163965,-1.525864,-0.281145,-0.069317,-0.120591,...,0.441693,-0.15436,-0.176145,-0.091248,0.210687,-0.033854,0.220808,-0.873854,0.170559,0.190579
Olfactory bulb (main) mitral cell adexp,-0.034168,-0.044056,-0.170394,-0.061382,0.318542,-1.155569,-1.527886,-0.331191,-0.359399,-0.120591,...,-0.590838,-0.15436,-0.176145,-0.083516,0.021426,-0.683607,0.050157,-0.873915,-0.437490,6.925665
Olfactory bulb (main) mitral cell fe_izhi,0.002683,-0.044053,-0.170396,-0.055735,-1.367810,-1.163013,-1.525865,-0.290434,-0.098648,-0.120591,...,0.194704,-0.15436,-0.176145,-0.052845,0.019219,-0.135920,0.220808,-0.873851,0.170559,0.434188


In [23]:
#print(len(experiment_df))
experiment_df = pd.DataFrame.drop_duplicates(experiment_df)
#experiment_df.index
experiment_df.groupby(experiment_df.index).first()
len(experiment_df)

448

In [24]:
print(len(model_df))
model_df = pd.DataFrame.drop_duplicates(model_df)
model_df.index
model_df.groupby(model_df.index).first()
len(model_df)

972


972

In [25]:
# Do PCA and look at variance explained
pca = PCA()
pca.fit(df.values)
n_features = df.shape[1]
plt.plot(range(1,n_features+1),pca.explained_variance_ratio_.cumsum())
plt.xlim(0,50);
plt.xlabel('Number of principal components')
plt.ylabel('Fraction of variance explained');
plt.show()

In [26]:
# THe transformed values, ordered from highest to lowest variance dimensions
transformed = pca.transform(df.values)


#model_idx = [idx for idx in enumerate(df.index.values) if type(idx)==str]
'''
#label_model_no_trans_df
model_no_trans_df.index.name = 'Cell_ID'
model_df = model_no_trans_df.copy()
model_df.index.name = 'Cell_ID'
'''
# make experiment dataframe
experiment_idx = [idx for idx in df.index.values if type(idx)==int]
model_no_trans_df = df[~df.index.isin(experiment_idx)]
experiment_idx_labels = [(i,idx) for i,idx in enumerate(df.index.values) if type(idx)==int]

#model_df
#df.labels
model_no_trans_df
experiment_idx_labels = [i[0] for i in experiment_idx_labels]
experiment_idx_labels
model_no_trans_df
model_index_labels = ~df.index.isin(experiment_idx)

model_index_labels


new_models = df.head(7).index.tolist()#  list(range(0,9))#idx for idx in df.index.values if type(idx)==str]
new_model_labels= df.index.isin(new_models)

In [27]:
# Do an isomap embedding in 2 dimensions
isomap = Isomap(n_components=2)
isomap.fit(df.values)
iso = isomap.embedding_.T
# Plot that isomap embedding. Each is a model (or a cell, for data)
#plt.scatter(iso);
plt.clf()
fig = plt.figure()
ax = plt.subplot(111)
plt.scatter(iso[0,experiment_idx_labels],iso[1,experiment_idx_labels],c='blue',cmap='rainbow',label='data')
plt.scatter(iso[0,model_index_labels],iso[1,model_index_labels],c='red',cmap='rainbow',label='models')
plt.scatter(iso[0,new_model_labels],iso[1,new_model_labels],c='green',cmap='rainbow',label='new models')

legend = ax.legend()#handles, labels, loc="upper right", title="Sizes")
# I don't love the isomap fit
plt.show()

In [28]:
# Do a TSNE embedding in two dimensions
tsne = TSNE(n_components=2, perplexity=30)
tsne.fit(df.values)
x = tsne.embedding_.T
#np.shape(x)
#print(experiment_idx)
# Plot that TSNE embedding. Each is a model (or a cell, for data)
plt.clf()
fig = plt.figure()
ax = plt.subplot(111)
plt.scatter(x[0,experiment_idx_labels],x[1,experiment_idx_labels],c='blue',cmap='rainbow',label='data')
plt.scatter(x[0,model_index_labels],x[1,model_index_labels],c='red',cmap='rainbow',label='models')
plt.scatter(x[0,new_model_labels],x[1,new_model_labels],c='green',cmap='rainbow',label='new models')

legend = ax.legend()#handles, labels, loc="upper right", title="Sizes")
plt.show()

* We can examine the dimension that contributes the most to cluster seperation  by looking at variance explained.


In [29]:
#df_data = pd.read_csv(...)
#df_models = pd.read_csv(...)
from sklearn.ensemble import RandomForestClassifier

df_models = df[~df.index.isin(experiment_idx)]
df_data = df[df.index.isin(experiment_idx)]
# Assume they have the same columns

df_combined = pd.concat((df_data, df_models))
#target = np.array([0]*df_models.shape[0] + [1]*df_data.shape[1])
target = np.array(df.index.isin(experiment_idx))

print(np.shape(df_combined))
print(np.shape(target))

rfc = RandomForestClassifier()
rfc.fit(df_combined.values, target)

importances = pd.Series(index = df_combined.columns, data=rfc.feature_importances_)
importances.sort_values(ascending=False)[0:10]
#rfc.feature_importances_

(1420, 38)
(1420,)


AP2AHPDepthTest                    0.174644
AP1AmplitudeTest                   0.151236
InputResistanceTest                0.126461
AP1RateOfChangePeakToTroughTest    0.105361
AP2RateOfChangePeakToTroughTest    0.085062
AP2AmplitudeTest                   0.082233
ISICVTest                          0.075434
Burst1ISIMeanTest                  0.072074
ISIBurstMeanChangeTest             0.057604
ISIMedianTest                      0.022663
dtype: float64

In [30]:
#np.shape(x)
#x[0,:]
#X[1,:]

In [31]:
# PLay around with the value of perplexity. Recommended values are between 5 and 50.  
#See if any of the clusters that pop out mean anything (do they match existing cell type clusters?  
# Do they match known firnig patterns?  
# Where does the data fit in when it is also plotted in this space?)

perplexities = [25,30,35,40]

fig, ax = plt.subplots(2,2,figsize=(8,12),dpi=300)
ax = ax.ravel()

for i, perp in enumerate(perplexities):

    # init = 'pca' or 'random'
    tsne = TSNE(n_components=2,
                init='random',      
                random_state=0,
                perplexity=perp,         # default = 30, should be less than the number of samples
                n_iter=1000)             # default = 1000


    %time tsne.fit(df.values) # can't use transpose


    ax[i].scatter(*tsne.embedding_.T);

    if i in [2,3]:
        ax[i].set_xlabel('TSNE-1')

    if i in [0,2]:
        ax[i].set_ylabel('TSNE-2')
    ax[i].set_title('Perplexity = %s' %perp)
plt.show()

CPU times: user 7.52 s, sys: 0 ns, total: 7.52 s
Wall time: 7.51 s
CPU times: user 8.15 s, sys: 116 ms, total: 8.26 s
Wall time: 8.02 s
CPU times: user 7.74 s, sys: 184 ms, total: 7.92 s
Wall time: 7.68 s
CPU times: user 11 s, sys: 204 ms, total: 11.2 s
Wall time: 10.9 s


### Repeat above with just experimental data

In [32]:
# make model dataframe
model_idx = [idx for idx in df.index.values if type(idx)==str]
model_no_trans_df = df[df.index.isin(model_idx)]
model_no_trans_df.index.name = 'Cell_ID'
model_df = model_no_trans_df.copy()
model_df.index.name = 'Cell_ID'

# make experiment dataframe
experiment_idx = [idx for idx in df.index.values if type(idx)==int]
experiment_no_trans_df = df[df.index.isin(experiment_idx)]
experiment_df = experiment_no_trans_df.copy()
experiment_df

,AP12AHPDepthPercentChangeTest,AP12AmplitudeChangePercentTest,AP12AmplitudeDropTest,AP12HalfWidthChangePercentTest,AP12RateOfChangePeakToTroughPercentChangeTest,AP1AHPDepthTest,AP1AmplitudeTest,AP1DelayMeanStrongStimTest,AP1DelayMeanTest,AP1DelaySDStrongStimTest,...,Burst1ISIMeanTest,Burst1ISISDStrongStimTest,Burst1ISISDTest,ISIBurstMeanChangeTest,ISICVTest,ISIMedianTest,InitialAccommodationMeanTest,InputResistanceTest,SSAccommodationMeanTest,SpikeRateStrongStimTest
313861539,-0.054288,-0.046613,-0.169249,-0.020866,-1.370341,-1.162504,-1.526790,-0.206645,-0.478425,-0.120591,...,2.592509,5.359894,4.588158,-0.244283,-0.083993,-0.639368,26.664188,-0.873915,22.668392,0.548828
313861677,-0.034168,-0.044255,-0.170241,-0.041103,0.318542,-1.163670,-1.526012,-0.012655,-0.478425,-0.120591,...,-0.036303,-0.154360,-0.176145,-0.058434,-0.076904,-0.242279,-0.513731,-0.873915,-0.437490,-0.712210
313862167,-0.034168,-0.043982,-0.170409,-0.052515,0.318542,-1.162743,-1.527366,-0.020143,-0.478425,-0.120591,...,-0.151765,-0.154360,-0.176145,-0.069105,-0.050328,-0.347360,0.220808,-0.873915,0.094553,-0.253651
313862167,-0.034168,-0.044056,7.195682,-0.061382,0.318542,-1.166949,0.173871,-0.154388,-0.478425,-0.120591,...,-0.623455,-0.154360,-0.176145,-0.083516,-0.084995,-0.711988,0.220808,-0.873915,0.170559,0.692128
314822529,-0.034168,-0.044056,-0.170394,-0.061382,0.318542,-1.166949,-1.526449,0.162189,-0.478425,-0.120591,...,-0.623455,-0.154360,-0.176145,-0.083516,-0.084995,-0.711988,-0.513731,-0.873915,-0.437490,-0.783860
318553131,-0.034168,-0.044098,-0.170382,-0.050696,0.318542,-1.166949,-1.527205,-0.155589,-0.478425,-0.120591,...,-0.498860,-0.154360,-0.176145,-0.056619,-0.025431,-0.622942,0.197853,-0.873915,0.151558,1.465946
318733871,-0.034168,-0.044056,-0.170394,-0.061382,0.318542,-1.166949,-1.526427,0.158081,-0.478425,-0.120591,...,-0.623455,-0.154360,-0.176145,-0.083516,-0.084995,-0.711988,-0.513731,-0.873915,-0.437490,-0.783860
320207387,-0.034168,-0.044056,-0.170394,-0.061382,0.318542,-1.163792,-1.526571,0.179787,-0.478425,-0.120591,...,-0.623455,-0.154360,-0.176145,-0.083516,-0.084995,-0.711988,-0.513731,-0.873915,-0.437490,-0.783860
320247631,-0.034168,-0.045078,-0.170014,-0.054606,0.318542,-1.162719,-1.526979,-0.140929,-0.478425,-0.120591,...,-0.623455,-0.154360,-0.176145,-0.083516,-0.084995,-0.613647,-0.513731,-0.873915,-0.437490,-0.769530
320455763,-0.038027,-0.043940,-0.170452,-0.055471,0.427053,-1.162219,-1.526648,-0.079383,-0.478425,-0.120591,...,-0.345780,-0.154360,-0.176145,-0.068104,-0.083539,-0.480681,0.037173,-0.873915,-0.184136,-0.253651


In [33]:
model_df[:] = ss.fit_transform(model_no_trans_df.values);

In [34]:
model_no_trans_df.head()

,AP12AHPDepthPercentChangeTest,AP12AmplitudeChangePercentTest,AP12AmplitudeDropTest,AP12HalfWidthChangePercentTest,AP12RateOfChangePeakToTroughPercentChangeTest,AP1AHPDepthTest,AP1AmplitudeTest,AP1DelayMeanStrongStimTest,AP1DelayMeanTest,AP1DelaySDStrongStimTest,...,Burst1ISIMeanTest,Burst1ISISDStrongStimTest,Burst1ISISDTest,ISIBurstMeanChangeTest,ISICVTest,ISIMedianTest,InitialAccommodationMeanTest,InputResistanceTest,SSAccommodationMeanTest,SpikeRateStrongStimTest
Cell_ID,,,,,,,,,,,,,,,,,,,,,
Cerebellum Purkinje cell adexp,-0.034168,-0.044056,-0.170394,-0.061382,0.318542,-1.161653,-1.527886,-0.337510,-0.393407,-0.120591,...,-0.601711,-0.15436,-0.176145,-0.083516,-0.061837,-0.697797,0.005640,-0.873915,-0.437490,7.083295
Cerebellum Purkinje cell fe_izhi,-0.034168,-0.044056,-0.170394,-0.061382,0.318542,-1.163067,-1.526587,-0.262662,-0.035224,-0.120591,...,-0.623455,-0.15436,-0.176145,-0.083516,-0.084995,-0.711988,-0.513731,-0.873789,-0.437490,-0.511590
Neocortex pyramidal cell layer 5-6 adexp,-0.034168,-0.044056,-0.170394,-0.061382,0.318542,-1.155569,-1.527886,-0.331191,-0.359399,-0.120591,...,-0.579966,-0.15436,-0.176145,-0.083516,-0.084995,-0.683607,0.042738,-0.873915,-0.437490,6.925665
Neocortex pyramidal cell layer 5-6 fe_izhi,-0.033224,-0.044054,-0.170395,-0.061382,0.404431,-1.161898,-1.525867,-0.290339,0.034321,-0.120591,...,0.300952,-0.15436,-0.176145,-0.081320,1.094775,-0.104630,0.073900,-0.873761,0.170559,0.161919
Hippocampus CA1 basket cell fe_izhi,-0.010420,-0.044058,-0.170392,-0.060397,3.410929,-1.163965,-1.525864,-0.281145,-0.069317,-0.120591,...,0.441693,-0.15436,-0.176145,-0.091248,0.210687,-0.033854,0.220808,-0.873854,0.170559,0.190579


In [35]:
model_df.head()

,AP12AHPDepthPercentChangeTest,AP12AmplitudeChangePercentTest,AP12AmplitudeDropTest,AP12HalfWidthChangePercentTest,AP12RateOfChangePeakToTroughPercentChangeTest,AP1AHPDepthTest,AP1AmplitudeTest,AP1DelayMeanStrongStimTest,AP1DelayMeanTest,AP1DelaySDStrongStimTest,...,Burst1ISIMeanTest,Burst1ISISDStrongStimTest,Burst1ISISDTest,ISIBurstMeanChangeTest,ISICVTest,ISIMedianTest,InitialAccommodationMeanTest,InputResistanceTest,SSAccommodationMeanTest,SpikeRateStrongStimTest
Cell_ID,,,,,,,,,,,,,,,,,,,,,
Cerebellum Purkinje cell adexp,-0.042261,0.625683,-0.644356,-0.159862,0.427858,-2.446992,-5.647891,-0.862787,-0.922515,0.0,...,-0.935727,-0.071907,-0.126041,-0.388189,-0.081316,-0.973826,-0.152470,-1.312184,-2.999728,6.874316
Cerebellum Purkinje cell fe_izhi,-0.042261,0.625683,-0.644356,-0.159862,0.427858,-2.449130,-5.644434,0.198520,-0.250072,0.0,...,-0.969391,-0.071907,-0.126041,-0.388189,-0.100504,-0.991670,-3.062415,-1.312054,-2.999728,-0.766713
Neocortex pyramidal cell layer 5-6 adexp,-0.042261,0.625683,-0.644356,-0.159862,0.427858,-2.437797,-5.647891,-0.773187,-0.858671,0.0,...,-0.902062,-0.071907,-0.126041,-0.388189,-0.100504,-0.955983,0.055383,-1.312184,-2.999728,6.715729
Neocortex pyramidal cell layer 5-6 fe_izhi,-0.041479,0.630564,-0.644361,-0.159862,0.543587,-2.447363,-5.642518,-0.193926,-0.119510,0.0,...,0.461784,-0.071907,-0.126041,-0.370969,0.877008,-0.227964,0.229979,-1.312026,1.071397,-0.089113
Hippocampus CA1 basket cell fe_izhi,-0.022597,0.621632,-0.644352,-0.028467,4.594600,-2.450488,-5.642510,-0.063559,-0.314076,0.0,...,0.679679,-0.071907,-0.126041,-0.448827,0.144487,-0.138969,1.053078,-1.312121,1.071397,-0.060279


In [36]:
# PLay around with the value of perplexity. Recommended values are between 5 and 50.  
#See if any of the clusters that pop out mean anything (do they match existing cell type clusters?  
# Do they match known firnig patterns?  
# Where does the data fit in when it is also plotted in this space?)

perplexities = [25,30,35,40]

df = model_df.copy()

fig, ax = plt.subplots(2,2,figsize=(8,12))
ax = ax.ravel()

for i, perp in enumerate(perplexities):

    # init = 'pca' or 'random'
    tsne = TSNE(n_components=2,
                init='random',      
                random_state=0,
                perplexity=perp,         # default = 30, should be less than the number of samples
                n_iter=1000)             # default = 1000


    %time tsne.fit(df.values) # can't use transpose


    ax[i].scatter(*tsne.embedding_.T);

    if i in [2,3]:
        ax[i].set_xlabel('TSNE-1')

    if i in [0,2]:
        ax[i].set_ylabel('TSNE-2')
    ax[i].set_title('Perplexity = %s' %perp)

CPU times: user 7.22 s, sys: 0 ns, total: 7.22 s
Wall time: 7.22 s
CPU times: user 7.83 s, sys: 160 ms, total: 7.99 s
Wall time: 7.75 s
CPU times: user 5.96 s, sys: 164 ms, total: 6.12 s
Wall time: 5.89 s
CPU times: user 11 s, sys: 184 ms, total: 11.2 s
Wall time: 11 s


In [37]:
try:
    os.mkdir('data')
except:
    pass
filename = os.path.join(path2data,'new_cortical_ephys.csv')

model_df.to_csv(filename)

filename = os.path.join(path2data,'new_cortical_ephys_no_trans.csv')
model_no_trans_df.to_csv(filename)

filename = os.path.join(path2data,'experiment_ephys_no_trans.csv')
experiment_no_trans_df.to_csv(filename)